# 3. Clases en Tkinter: Ejemplo

Veamos un programa de ejemplo que crea una ventana con un editor de texto y varias funcionalidades como imprimir el contenido, limpiar el área de texto, deshacer y rehacer cambios. Usa una `deque` para gestionar el historial de ediciones y permite que el usuario interactúe con la interfaz gráfica de manera eficiente.

## 3.1 ¿Qué es una *deque* en Python?
Una **`deque`** (double-ended queue o cola de dos extremos) en Python es una estructura de datos que permite agregar y eliminar elementos de ambos extremos de manera eficiente, es decir, desde el principio o el final de la cola. A diferencia de una lista (`list`), que es más lenta para operaciones de inserción o eliminación al principio, una `deque` está optimizado para estas tareas. Además, podemos definir su tamaño máximo con el parámetro `maxlen`, lo que resulta útil para implementar colas con longitud limitada (como un historial de cambios), eliminándose automáticamente los elementos más antiguos cuando se supera el límite. Las operaciones más usuales con una `deque`serán: `append`, `appendleft`, `pop` y `popleft`. Ejemplo:

In [ ]:
from collections import deque

# Crear un deque vacío con un tamaño máximo de 5
d = deque(maxlen=5)

# Agregar elementos al final
d.append(1)
d.append(2)
d.append(3)

# Agregar un elemento al principio
d.appendleft(0)

# Eliminar elementos desde el final y el principio
d.pop()       # Elimina 3
d.popleft()   # Elimina 0

print(d)  # deque([1, 2])

## 3.2 ¿Por qué usar clases con Tkinter?
El uso de clases facilita la organización del código y el manejo de las interacciones de usuario. La clase `Window` encapsula todos los elementos de la ventana y la lógica asociada. Esto facilita que, más adelante, podamos extender la funcionalidad del programa o mejorar las existentes. Las clases permiten tener una instancia de la ventana con todas sus propiedades y métodos, haciendo que el código sea más reutilizable y fácil de mantener.

## 3.3 Código del ejemplo:

In [ ]:
from tkinter import *
from collections import deque


class Window:
    def __init__(self, master):
        self.master = master

        self.Main = Frame(self.master)

        self.stack = deque(maxlen=10)
        self.stackcursor = 0

        self.L1 = Label(self.Main, text="This is my NotePad")
        self.L1.pack(padx=5, pady=5)

        self.T1 = Text(self.Main, width=80, height=20)
        self.T1.pack(padx=5, pady=5)

        self.menu = Menu(self.Main)
        self.menu.add_command(label="Print", command=self.print_stack)
        self.menu.add_command(label="Undo", command=self.undo)
        self.menu.add_command(label="Redo", command=self.redo)
        self.master.config(menu=self.menu)

        self.B1 = Button(self.Main, text="Print", width=8, command=self.display)
        self.B1.pack(padx=5, pady=5, side="left")

        self.B2 = Button(self.Main, text="Clear", width=8, command=self.clear)
        self.B2.pack(padx=5, pady=5, side="left")

        self.B3 = Button(self.Main, text="Undo", width=8, command=self.undo)
        self.B3.pack(padx=5, pady=5, side="left")

        self.B4 = Button(self.Main, text="Redo", width=8, command=self.redo)
        self.B4.pack(padx=5, pady=5, side="left")

        self.Main.pack(padx=5, pady=5)

    def display(self):
        print(self.T1.get("1.0", "end"))

    def clear(self):
        self.T1.delete("1.0", "end")

    def stackify(self):
        self.stack.append(self.T1.get("1.0", "end - 1c"))
        if self.stackcursor < 9: self.stackcursor += 1

    def undo(self):
        if self.stackcursor != 0:
            self.clear()
            if self.stackcursor > 0: self.stackcursor -= 1
            self.T1.insert("0.0", self.stack[self.stackcursor])

    def redo(self):
        if len(self.stack) > self.stackcursor + 1:
            self.clear()
            if self.stackcursor < 9: self.stackcursor += 1
            self.T1.insert("0.0", self.stack[self.stackcursor])

    def print_stack(self):
        i = 0
        for stack in self.stack:
            print(str(i) + " " + stack)
            i += 1


root = Tk()
window = Window(root)
root.bind("<Key>", lambda event: window.stackify())
root.mainloop()

## 3.4 Análisis del código:

1. **Importación de la clase `deque`:**

   * `from collections import deque`: Importa la clase `deque` de la librería `collections` que, en este caso, se usará para almacenar el historial de texto para las funcionalidades de deshacer/rehacer).

2. **Clase `Window`:**: Es el núcleo de la aplicación. Al ser instanciada, crea una ventana principal de la interfaz con widgets de Tkinter.

   * **`__init__(self, master)`**: Constructor de la clase `Window`. Recibe el parámetro `master`, que es la ventana principal creada por `Tk()`.
     * `self.master = master`: Se guarda una referencia a la ventana principal.
     * `self.Main = Frame(self.master)`: Se crea un `Frame` dentro de la ventana principal para organizar los widgets.
     * `self.stack = deque(maxlen=10)`: Inicializa una `deque` que almacenará hasta 10 versiones previas del contenido del área de texto.
     * `self.stackcursor = 0`: Mantiene el índice actual del historial de ediciones.

3. **Métodos de la clase `Window`:**

   * **`display(self)`**: Imprime el contenido actual del área de texto en la consola.
   * **`clear(self)`**: Limpia el área de texto.
   * **`stackify(self)`**: Guarda el contenido actual del área de texto en el historial (`stack`). Solo guarda las versiones cuando el contenido cambia, y limita el tamaño del historial a 10 entradas.
   * **`undo(self)`**: Revierte al estado anterior en el historial de ediciones (si hay un historial disponible). La función también actualiza el `stackcursor` para señalar el índice del historial actual.
   * **`redo(self)`**: Restaura el estado siguiente en el historial (si está disponible). Similar al funcionamiento de `undo`.
   * **`print_stack(self)`**: Imprime el contenido completo del historial (`stack`), mostrando las versiones previas del texto.

4. **Configuración de la ventana principal (`root`):**

   * **`root = Tk()`**: Crea la ventana principal.
   * **`window = Window(root)`**: Crea una instancia de la clase `Window`, pasando la ventana principal como argumento.
   * **`root.bind("<Key>", lambda event: window.stackify())`**: Asocia la función `stackify` a cada tecla presionada.
   * **`root.mainloop()`**: Inicia el bucle principal de la aplicación para que la interfaz gráfica permanezca activa y espere interacciones del usuario.

## 3.4 Ejercicios

### **Ejercicio 1. Restringir el contenido de texto** (uso de `if-elif-else`)
Agregar una funcionalidad que limite ciertos caracteres o palabras en el área de texto, por ejemplo, evitar que el usuario escriba palabras ofensivas o cadenas específicas.

Por ejemplo, supongamos que quieres restringir el uso de ciertas palabras ("Python", "error"). Puedes usar un `if-elif-else` para verificar si el texto contiene alguna de esas palabras y evitar que se muestre en el área de texto. Usa `text_content = self.T1.get("1.0", "end - 1c")`para obtener el texto del widget. Cuando el usuario intente escribir palabras restringidas, el contenido no se guarda en el historial, y se mostrará un mensaje en la consola.


### **Ejercicio 2. Añadir una funcionalidad para cambiar el color de fondo** (uso de `if-elif-else`)
Añadir botones que permitan al usuario cambiar el color de fondo del área de texto (por ejemplo, "Blanco", "Negro", "Azul"). Implementa una nueva función que contenga este código y que podamos referenciar desde los botones al pulsarlos.


### **Ejercicio 3. Controlar el tamaño del área de texto** (uso de `match-case`)
Ofrece que el tamaño del área de texto pueda cambiar dependiendo de la cantidad de texto que se haya escrito. Usa una sentencia `match-case` para ajustar dinámicamente el tamaño del área de texto según el contenido.
Por ejemplo:
* Si el texto tiene menos de 50 caracteres, el área de texto es más pequeña.
* Si el texto tiene entre 50 y 200 caracteres, el área de texto es mediana.
* Si el texto tiene más de 200 caracteres, el área de texto es grande.

De nuevo, puedes obtener el texto del widget empleando `text_content = self.T1.get("1.0", "end-1c")`. Usa la función `len()` para hallar su longitud.
Puedes llamar a este método después de cada cambio en el texto, dentro de `stackify` o en cualquier otro momento apropiado.

### **Ejercicio 4. Confirmación antes de limpiar el área de texto** (uso de `if-else`)
Añade una confirmación antes de limpiar el área de texto (para evitar pérdidas accidentales de datos), puedes hacerlo con una simple sentencia condicional `if-else`. Emplea la función `askyesno` del módulo `messagebox` de Tkinter para pedir confirmación al usuario antes de borrar el contenido. Si el usuario selecciona "Sí", se limpia el área de texto (`self.T1.delete("1.0", "end")`), y si selecciona "No", no se hace nada (`pass`).

## 3.5 Complemento: getter y setters en las clases de Python

En Python, el uso de getters y setters no es tan común como en otros lenguajes como Java o C#. En ellos, los getters y setters se utilizan para encapsular el acceso a los atributos de una clase. Esto es útil cuando es necesario aplicar ciertas reglas de validación, transformación, o control de acceso a los atributos de la clase.

En Python, la filosofía del lenguaje a este respecto se resume en la frase "we are all consenting adults here" que, en esencia, significa que se asume que los programadores son responsables y pueden tomar decisiones informadas sobre cómo manipulan los datos y los objetos. No es necesario protegerlos de todo ni imponer reglas estrictas sobre el acceso a los atributos, ya que se confía en que los programadores seguirán buenas prácticas o serán conscientes de las implicaciones de sus decisiones.

Por eso, en Python, acceder directamente a los atributos de una clase es completamente aceptado y generalmente preferido, a menos que necesites (como decíamos antes) aplicar algún tipo de validación o transformación, cuando trabajes con bibliotecas o APIs que requieran de un estilo más tradicional de acceso a los atributos o cuando escribas tu propia API y quieres mantener cierta compatibilidad con otros lenguajes.

En estos casos, las forma más "pythónica" de hacerlo es usando "propiedades". Las propiedades permiten definir métodos para obtener y establecer un atributo (generalmente privado), pero se usan como si fueran atributos directos. Son más elegantes y flexibles que los getters y setters tradicionales. Ejemplo:

```python
class Persona:
    def __init__(self, nombre):
        self._nombre = nombre  # Usamos un atributo privado

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, valor):
        if not valor:
            raise ValueError("El nombre no puede estar vacío")
        self._nombre = valor

p = Persona("Juan")
print(p.nombre)  # Usa el getter, pero parece un atributo
p.nombre = "Pedro"  # Usa el setter, pero parece un atributo
```
De esta forma no es necesario llamar a un método explícitamente como en otros lenguajes. El acceso y la modificación de los valores se ve como si fueran atributos directos.

Con todo, también es posible realizar validaciones sobre atributos públicos:
```python
class Persona:
    def __init__(self, nombre):
        if not nombre:
            raise ValueError("El nombre no puede estar vacío")
        self.nombre = nombre

p = Persona("Juan")
print(p.nombre)
```
En este ejemplo, la validación de que el nombre no esté vacío se hace directamente en el constructor, sin necesidad de usar getters y setters adicionales.